In [7]:
import sys
sys.path.append('../')
from time import time
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm.session import Session
import pandas as pd

def conn_engine(username: str, password: str, host: str, db_name: str, **_kwargs):
    db_url = f"mysql+pymysql://{username}:{password}@{host}:3306/{db_name}"
    return create_engine(db_url)

def connect_db(username: str, password: str, host: str, db_name: str, **_kwargs) -> Session:

    engine = conn_engine(username, password, host, db_name)
    session_local = sessionmaker(bind=engine)
    return session_local()

secret = {
        "username": "admin",
        "password": "1q2w3e4r!",
        "host": "dodomoards.ccoalf3s8d7c.ap-northeast-2.rds.amazonaws.com",
        "db_name": "captured",
    }

def execute_query(query):
    """esecute_query"""
    db_session = connect_db(**secret)
    result_list = db_session.execute(query)
    if db_session.is_active:
        db_session.close()
    return list(result_list)

s_1 = time()
x = execute_query("SELECT * FROM product_info ")
s_2 = time()

print("query executing", f"{s_2 - s_1:.5f} sec")


query executing 0.18492 sec


### update or insert Query

In [2]:
from tables import ProductInfoTable, SizeTable
from sqlalchemy import func,case
session = connect_db(**secret)

### update

In [10]:
df = pd.DataFrame(x)

In [11]:
# session.rollback()

In [21]:
price_asc_cursor = [str(10**11-int(str(y[0]).zfill(7) + str(y[1]).zfill(5))) for y in df[['price','sku']].values]
price_desc_cursor = [(str(y[0]).zfill(7) + str(y[1]).zfill(5)) for y in df[['price','sku']].values]

In [22]:
df['price_asc_cursor'] = price_asc_cursor
df['price_desc_cursor'] = price_desc_cursor

In [25]:
session.bulk_update_mappings(ProductInfoTable, df.to_dict(orient='records'))
session.commit()